In [1]:
import os
import sys
sys.path.append('/'.join(os.getcwd().split('/')[:-1]))

In [2]:
from model.bAbI_utils import bAbI_data_loader, data_loader, pad_to_batch, pad_to_story

test: train_loader

In [3]:
path = '../data/QA_bAbI_tasks/en-10k/qa1_single-supporting-fact_train.txt'
data, word2idx = bAbI_data_loader(path)

In [4]:
batch_size = 32
for batch in data_loader(data, batch_size, shuffle=False):
    break

In [5]:
stories, stories_masks, questions, questions_masks, answers, supports = pad_to_batch(batch, word2idx)

In [6]:
stories.size()

torch.Size([32, 10, 7])

In [7]:
stories_masks.size()

torch.Size([32, 10, 7])

In [8]:
questions.size()

torch.Size([32, 4])

In [9]:
questions_masks.size()

torch.Size([32, 4])

In [10]:
answers.size()

torch.Size([32, 2])

test: test_loader

In [11]:
path_test = '../data/QA_bAbI_tasks/en-10k/qa1_single-supporting-fact_test.txt'
data_test, word2idx = bAbI_data_loader(path_test, word2idx)

In [12]:
batch_size = 32
for batch in data_loader(data_test, batch_size, shuffle=False):
    break

In [13]:
stories, stories_masks, questions, questions_masks, answers, supports = pad_to_batch(batch, word2idx)

In [14]:
stories.size()

torch.Size([32, 10, 7])

In [15]:
stories_masks.size()

torch.Size([32, 10, 7])

In [16]:
questions.size()

torch.Size([32, 4])

In [17]:
questions_masks.size()

torch.Size([32, 4])

In [18]:
answers.size()

torch.Size([32, 2])

test: pad_to_story

In [19]:
stories, stories_masks, questions, questions_masks, answers, supports = pad_to_story(data_test, word2idx)

In [20]:
len(stories), len(stories_masks)

(1000, 1000)

In [21]:
len(questions), len(questions_masks)

(1000, 1000)

In [22]:
len(answers), len(supports)

(1000, 1000)